In [8]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

In [2]:
def faculty_members():
    names =  ['นพ.สมเกียรติ ลีละศิธร',
    'พญ.วัชรา ริ้วไพบูลย์',
    'ศ.ดร.ทวี  เชื้อสุวรรณทวี',
    'รศ.ดร.อาดัม นีละไพจิตร',
    'รศ.ดร.ณัฏฐนียา โตรักษา',
    'ผศ.ดร.อารี ภาวสุทธิไพศิฐ',
    'ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล',
    'ผศ.ดร.เจนจิรา เจนจิตรวาณิช',
    'ดร.รติรส จันทรสมดี',
    'ดร.อิศวรา ศิริรุ่งเรือง',
    'ดร.ธรรม จตุนาม',
    'ดร.รุจิรา สงขาว',
    'ดร.ปรเมศวร์ บุญยืน',
    'ดร.ปกรณ์กิตติ์ ม่วงประสิทธิ์',
    'ดร.สุนันทา ขลิบทอง',
    'ดร.วรางคณา รัชตะวรรณ',
    'ดร.ญาณิศา เนียรนาทตระกูล',
    'ดร.สุภชาญ ตรัยตรึงศ์สกุล',
    'ดร.วิษณุ นิตยธรรมกุล',
    'พฤหัส ศุภจรรยา',
    'ราษฏร์ บุญญา',
    'กุลยา ไทรงาม',
    'ณัฐวิชญ์ ศุภสินธุ์',
    'พชร นิลมณี',
    'สิรินทรา ฤทธิเดช',
    'สร้อยทอง หยกสุริยันต์',
    'ภัทรานิษฐ สงประชา']
    return names

# option = st.selectbox(
#     "ผู้สอนหลัก",
#     faculty_members(),
# )

# st.write("You selected:", option)

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# # drop down lecturer name
# import streamlit as st

# option = st.selectbox(
#     "ผู้สอนหลัก",
#     faculty_members(),
# )

# st.write("You selected:", option)


In [3]:
mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")

In [ ]:
academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '6/1/2025'
    end_date   = '12/10/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

In [4]:
start_date = '12/12/2025'
end_date   = '12/13/2025'
f = F[F['Timestamp'].between(start_date, end_date)]

In [5]:
t1,t2 = report_mm3(f)

2025-12-16 09:15:31.898 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 09:15:32.486 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 09:15:32.926 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-16 09:15:32.927 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 09:15:32.928 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 09:15:32.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-16 09:15:33.545 Thread 'MainThread': missing ScriptRunContext! This warning can b

In [7]:
t2

,name,จำนวนวิชาที่รับผิดชอบ,จำนวนวิชาที่สอน
0,นพ.สมเกียรติ ลีละศิธร,0.0,0.0
1,พญ.วัชรา ริ้วไพบูลย์,-0.0,1.0
2,ศ.ดร.ทวี เชื้อสุวรรณทวี,-1.0,1.0
3,รศ.ดร.อาดัม นีละไพจิตร,-1.0,4.0
4,รศ.ดร.ณัฏฐนียา โตรักษา,-0.0,1.0
5,ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล,-1.0,1.0
6,ผศ.ดร.เจนจิรา เจนจิตรวาณิช,-0.0,1.0
7,ดร.รติรส จันทรสมดี,-3.0,3.0
8,ดร.อิศวรา ศิริรุ่งเรือง,-2.0,2.0
9,ดร.ธรรม จตุนาม,-1.0,0.0
